In [ ]:
import base64
import json
import os
import requests

In [ ]:
data_path = os.getenv('DATA_PATH')
file_path = lambda *args: os.path.join(data_path, 'diygenomics-projects', 'experiment-a', *args)

file_name = '2021_Wightman-Posthuma_A_genomewide_association_study_with_112_563_individuals_identifies_new_risk_loci_for_Alzheimers_disease.pdf'

endpoint = 'https://api.mathpix.com/v3/'
extensions = ['.mmd', '.docx', '.tex', '.html', '.lines.json', '.lines.mmd.json']
app_id = os.getenv('MATHPIX_APP_ID')
app_key = os.getenv('MATHPIX_APP_KEY')

headers = {
    'app_id': app_id,
    'app_key': app_key,
}

conversion_options = {
    'conversion_formats': {'docx': True, 'tex.zip': True},
    'math_inline_delimiters': ['$', '$'],
    'rm_spaces': True
}

base_name = os.path.splitext(os.path.basename(file_name))[0].replace(' ', '_')

if not os.path.exists(file_path(base_name)):
    os.mkdir(file_path(base_name))
    
if not os.path.exists(file_path(base_name, 'mathpix')):
    os.mkdir(file_path(base_name, 'mathpix'))

In [ ]:
def download_file(pdf_id, extension):
    file_with_extension = f'{pdf_id}{extension}'
    url = os.path.join(endpoint, 'pdf', file_with_extension) 
    response = requests.get(url, headers=headers)
    if extension == '.tex':
        file_with_extension = f'{file_with_extension}.zip'
        
    if extension == '.mmd':
        with open(file_path(base_name, 'mathpix', file_with_extension), "w") as f:
                f.write(response.text) 
    else:
         with open(file_path(base_name, 'mathpix', file_with_extension), "wb") as f:
                f.write(response.content)        

In [ ]:
r = requests.post(os.path.join(endpoint, 'pdf'),
    headers=headers,
    data={
        'options_json': json.dumps(conversion_options)
    },
    files={
        'file': open(file_path(file_name), 'rb')
    }
)

In [ ]:
pdf_id = r.json()['pdf_id']

In [ ]:
# check conversion status
# TODO wait until complete then next step
r = requests.post(os.path.join(endpoint, 'converter', pdf_id),
    headers=headers
)

In [ ]:
for extension in extensions:
    download_file(pdf_id, extension)